In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import cv2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: #4169E1;"><b style="color:white;">Control Crown-of-Thorns Starfish</b></h1></center>


Crown-of-Thorns Starfish

"Since 1962, crown-of-thorns starfish outbreaks have had a major impact on the many reefs that make up the Great Barrier Reef. A fourth outbreak is currently underway in the World-Heritage Area."

"Crown-of-thorns starfish (also known as COTS) are marine invertebrates that feed on coral. They occur naturally on reefs throughout the Indo-Pacific region, and when conditions are right, they can reach plague proportions and devastate hard coral communities."

"Laboratory research at AIMS (Australian Institute of Marine Science) has shown that survival of crown-of-thorns starfish larvae increases dramatically when phytoplankton, their food source, becomes more abundant. Phytoplankton numbers are usually low in reef waters, but production can increase rapidly if early-season monsoonal and cyclonic floods carry fertilisers and other pollutants into the Great Barrier Reef lagoon."

"Once dense breeding populations of starfish develop on some reefs, the huge numbers of larvae that they produce can establish outbreaks on mid-shelf reefs in the central Reef, even though these reefs are hardly ever affected by runoff."

https://www.aims.gov.au/docs/research/biodiversity-ecology/threats/cots.html

#Surviving corals eaten by Crown of Thorns Starfish | WWF-Australia

https://www.youtube.com/watch?v=c9-QGMUUbbU  - 10 de jan. de 2019

Scientists fear starfish could combine with bleaching in “perfect storm” of Reef destruction. 

A new WWF-Australia report urges a crackdown on the “excessive, and often illegal,” use of industrial fertilisers that trigger outbreaks.
–

MORE FROM WWF-AUSTRALIA

SOCIAL:
Facebook... ►https://www.facebook.com/wwfaustralia/
Instagram.. ► https://www.instagram.com/wwf_australia/
Twitter........ ► https://twitter.com/WWF_Australia/

<iframe width="956" height="538" src="https://www.youtube.com/embed/c9-QGMUUbbU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [ ]:
os.listdir('../input/tensorflow-great-barrier-reef/')

In [ ]:
train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv', index_col=0)
train.head()

In [ ]:
test = pd.read_csv('../input/tensorflow-great-barrier-reef/test.csv', index_col=0)
test.head()

![](https://www.gbrmpa.gov.au/__data/assets/image/0010/267229/varieties/max1300.jpg)gbrmpa.gov.au

In [ ]:
!pip install kornia

#It's a TensorFlow Competition.  My bad, I'm sorry.

In [ ]:
import torch
import torchvision
import kornia as K

In [ ]:
img_bgr: np.array = cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_1/10004.jpg')  # HxWxC / np.uint8
img_rgb: np.array = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

plt.imshow(img_rgb); plt.axis('off');

#Load an image with Torchvision

It returns the images in a torch.Tensor in the shape (C,H,W).

In [ ]:
x_rgb: torch.tensor = torchvision.io.read_image('../input/tensorflow-great-barrier-reef/train_images/video_1/10004.jpg')  # CxHxW / torch.uint8
x_rgb = x_rgb.unsqueeze(0)  # BxCxHxW
print(x_rgb.shape);

#Load an image with Kornia

"The utility is kornia.image_to_tensor which casts a numpy.ndarray to a torch.Tensor and permutes the channels to leave the image ready for being used with any other PyTorch or Kornia component. The image is casted into a 4D torch.Tensor with zero-copy."

https://kornia-tutorials.readthedocs.io/en/latest/hello_world_tutorial.html

In [ ]:
#Code by https://kornia-tutorials.readthedocs.io/en/latest/hello_world_tutorial.html

x_bgr: torch.tensor = K.image_to_tensor(img_bgr)  # CxHxW / torch.uint8
x_bgr = x_bgr.unsqueeze(0)  # 1xCxHxW
print(f"convert from '{img_bgr.shape}' to '{x_bgr.shape}'")

#Convert from BGR to RGB with a kornia.color component.

In [ ]:
x_rgb: torch.tensor = K.color.bgr_to_rgb(x_bgr)  # 1xCxHxW / torch.uint8

#Visualize an image with Matplotib

In [ ]:
img_bgr: np.array = K.tensor_to_image(x_bgr)
img_rgb: np.array = K.tensor_to_image(x_rgb)

In [ ]:
#Code by https://kornia-tutorials.readthedocs.io/en/latest/hello_world_tutorial.html

fig, axs = plt.subplots(1, 2, figsize=(32, 16))
axs = axs.ravel()

axs[0].axis('off')
axs[0].imshow(img_rgb)

axs[1].axis('off')
axs[1].imshow(img_bgr)

plt.show()

In [ ]:
#Code by https://kornia-tutorials.readthedocs.io/en/latest/data_augmentation_sequential.html?highlight=bbox

from kornia import augmentation as K
from kornia.augmentation import AugmentationSequential
#from kornia.geometry import bbox_to_mask   #Deprecated??
from kornia.utils import image_to_tensor, tensor_to_image
from torchvision.transforms import transforms
from kornia.geometry.bbox import bbox_to_mask as _bbox_to_mask

to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

def plot_resulting_image(img, bbox, keypoints, mask):
    img = img * mask
    img_draw = cv2.polylines(np.array(to_pil(img)), bbox.numpy(), isClosed=True, color=(255, 0, 0))
    for k in keypoints[0]:
        img_draw = cv2.circle(img_draw, tuple(k.numpy()[:2]), radius=6, color=(255, 0, 0), thickness=-1)
    return img_draw

img = cv2.imread("../input/tensorflow-great-barrier-reef/train_images/video_1/10004.jpg", cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]

img_tensor = image_to_tensor(img).float() / 255.
plt.imshow(img); plt.axis('off');

#Define Augmentation Sequential and Different Labels

In [ ]:
#Code by https://kornia-tutorials.readthedocs.io/en/latest/data_augmentation_sequential.html?highlight=bbox

aug_list = AugmentationSequential(
    K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=1.0),
    K.RandomAffine(360, [0.1, 0.1], [0.7, 1.2], [30., 50.], p=1.0),
    K.RandomPerspective(0.5, p=1.0),
    data_keys=["input", "bbox", "keypoints", "mask"],
    return_transform=False,
    same_on_batch=False,
)

bbox = torch.tensor([[[355,10],[660,10],[660,250],[355,250]]])
keypoints = torch.tensor([[[465, 115], [545, 116]]])
mask = bbox_to_mask(torch.tensor([[[155,0],[900,0],[900,400],[155,400]]]), w, h).float()

img_out = plot_resulting_image(img_tensor, bbox, keypoints, mask)
plt.imshow(img_out); plt.axis('off');

I tried from kornia.geometry.bbox import bbox_to_mask as _bbox_to_mask  Though the warning above persists. 

#Forward Computations

In [ ]:
#Code by https://kornia-tutorials.readthedocs.io/en/latest/data_augmentation_sequential.html?highlight=bbox

out_tensor = aug_list(img_tensor, bbox.float(), keypoints.float(), mask)
img_out = plot_resulting_image(
    out_tensor[0][0],
    out_tensor[1].int(),
    out_tensor[2].int(),
    out_tensor[3][0],
)
plt.imshow(img_out); plt.axis('off');

#Inverse Transformations

In [ ]:
#Code by https://kornia-tutorials.readthedocs.io/en/latest/data_augmentation_sequential.html?highlight=bbox

out_tensor_inv = aug_list.inverse(*out_tensor)
img_out = plot_resulting_image(
    out_tensor_inv[0][0],
    out_tensor_inv[1].int(),
    out_tensor_inv[2].int(),
    out_tensor_inv[3][0],
)
plt.imshow(img_out); plt.axis('off');

In [ ]:
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()


def plot_resulting_image(img, bbox, keypoints, mask):
    img = img * mask
    img_draw = cv2.polylines(np.array(to_pil(img)), bbox.numpy(), isClosed=True, color=(255, 0, 0))
    for k in keypoints[0]:
        img_draw = cv2.circle(img_draw, tuple(k.numpy()[:2]), radius=6, color=(255, 0, 0), thickness=-1)
    return img_draw

img = cv2.imread("../input/tensorflow-great-barrier-reef/train_images/video_2/100.jpg", cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]

img_tensor = image_to_tensor(img).float() / 255.
plt.imshow(img); plt.axis('off');

#Patch Augmentation Sequential with patchwise_apply=True

In [ ]:
#Code by https://kornia-tutorials.readthedocs.io/en/latest/data_patch_sequential.html

from kornia.augmentation import PatchSequential, ImageSequential

pseq = PatchSequential(
    ImageSequential(
        K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=0.5),
        K.RandomPerspective(0.2, p=0.5),
        K.RandomSolarize(0.1, 0.1, p=0.5),
    ),
    K.RandomAffine(15, [0.1, 0.1], [0.7, 1.2], [0., 20.], p=0.5),
    K.RandomPerspective(0.2, p=0.5),
    ImageSequential(
        K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=0.5),
        K.RandomPerspective(0.2, p=0.5),
        K.RandomSolarize(0.1, 0.1, p=0.5),
    ),
    K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=0.5),
    K.RandomAffine(15, [0.1, 0.1], [0.7, 1.2], [0., 20.], p=0.5),
    K.RandomPerspective(0.2, p=0.5),
    K.RandomSolarize(0.1, 0.1, p=0.5),
    K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=0.5),
    K.RandomAffine(15, [0.1, 0.1], [0.7, 1.2], [0., 20.], p=0.5),
    ImageSequential(
        K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=0.5),
        K.RandomPerspective(0.2, p=0.5),
        K.RandomSolarize(0.1, 0.1, p=0.5),
    ),
    K.RandomSolarize(0.1, 0.1, p=0.5),
    K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=0.5),
    K.RandomAffine(15, [0.1, 0.1], [0.7, 1.2], [0., 20.], p=0.5),
    K.RandomPerspective(0.2, p=0.5),
    K.RandomSolarize(0.1, 0.1, p=0.5),
    patchwise_apply=True,
    same_on_batch=True,
)
out_tensor = pseq(img_tensor[None].repeat(2, 1, 1, 1))
to_pil(torch.cat([out_tensor[0], out_tensor[1]], dim=2))

#Patch Augmentation Sequential with patchwise_apply=False

In [ ]:
#Code by https://kornia-tutorials.readthedocs.io/en/latest/data_patch_sequential.html

pseq = PatchSequential(
    K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=0.75),
    K.RandomElasticTransform(alpha=(4., 4.)),
    patchwise_apply=False,
    same_on_batch=False
)
out_tensor = pseq(img_tensor[None].repeat(2, 1, 1, 1))
to_pil(torch.cat([out_tensor[0], out_tensor[1]], dim=2))

#Acknowledgement:

@inproceedings{eriba2020kornia, author = {E. Riba, D. Mishkin, J. Shi, D. Ponsa, F. Moreno-Noguer and G. Bradski}, title = {A survey on Kornia: an Open Source Differentiable Computer Vision Library for PyTorch}, year = {2020}, }

@inproceedings{eriba2019kornia, author = {E. Riba, D. Mishkin, D. Ponsa, E. Rublee and G. Bradski}, title = {Kornia: an Open Source Differentiable Computer Vision Library for PyTorch}, booktitle = {Winter Conference on Applications of Computer Vision}, year = {2020}, url = {https://arxiv.org/pdf/1910.02190.pdf} }

@misc{Arraiy2018, author = {E. Riba, M. Fathollahi, W. Chaney, E. Rublee and G. Bradski}, title = {torchgeometry: when PyTorch meets geometry}, booktitle = {PyTorch Developer Conference}, year = {2018}, url = {https://drive.google.com/file/d/1xiao1Xj9WzjJ08YY_nYwsthE-wxfyfhG/view?usp=sharing} }

#Though it's TensorFlow Competition, I made a Kaggle Notebook with a library that uses PyTorch (awkward situation).